# MIRAC-5 Reduce Helper

This jupyter notebook is intended to both provide instructions on how to use the mirac5reduce package and serve as a simple user interface for performing their own simple reductions.

To use this notebook, always start with the first two sections:

1. [Set Up the Config File](#setup-config)

2. [Set Up Python Environment](#setup-pyenv)

The remaining sections can be accessed as needed to perform various tasks:

3. [Calculate Mean Dark File](#calc-meandark)

4. [Create Bad Pixel Mask](#calc-bpmask)

5. [Create Flatfield File](#calc-flatfield)

6. [Create Mean Flat File](#calc-meanflat)

7. [Reduce Chop/Nod Observations](#reduce-chopnod)

8. [Reduce Staring Mode Observation](#reduce-staring)


<a id="setup-config"></a>
## Set Up the Config File

First, you will want to create your own copy of the config file and edit it. 

The sample config file is provided as part of the mirac5reduce package under `docs/runparams.init`. 

Create a copy of that file, preferably somewhere near where your reduction files will be, though that isn't required. 

Enter the path and file name of this copy in the following cell to save it as the variable, `config_file`:

In [ ]:
config_file = 

Open your config file in your text editor of choice. You will see that it has 4 sections: `[REDUCTION]`, `[CALIB]`, `[COMPUTING]`, and `[DATA_ARCH]`.

The last two sections, `[COMPUTING]` and `[DATA_ARCH]` contain parameters that are specific to the computer that you are running the reduction on and the architecture of the data produced by the MIRAC-5 data aqcuisition software, respectively. 

**Review and set the parameters in the `[COMPUTING]` and `[DATA_ARCH]` sections of your config file:** 

- Make sure that the `raw_name_fmt` parameter in the `[DATA_ARCH]` section matches the naming convention of the raw files from the telescope.



- In the `[COMPUTING]` section, we also recommend enabling memory saving capabilities unless you are on an extremely high-memory machine or are working with unusually small data sets for the instrument. To enable memory saving mode, set `save_mem` in this section to True and set the maximum number of frames instantaneously loaded into the memory with the `max_frames_inmem` key. (The best value depends on your system's memory, but a good starting point is to set this to be around 100-200.)

They should not have to change for the rest of the reduction. However, if you find computations are eating up too much memory, you may need to come back and adjust your `max_frames_inmem` setting. Descriptions are provided in the template config file. 

The remaining sections, `[REDUCTION]` and `[CALIB]` contain parameters that can either be provided directly when calling a function or imported from this config file to simplify function calls.

To aid in flexibility, this notebook will allow all of the parameters in these sections to be provided directly when calling the functions. 

**Make sure all other keyword values in these two sections are empty and save your config file.**


<a id="setup-pyenv"></a>
## Set Up Python Environment

Briefly, we import the scripts that contain the main upper-level functions:

In [ ]:
from mirac5reduce.reduce import combine_frames
from mirac5reduce.cal import bpmask, flatfield

In addition, all of the upper-level functions in `mirac5reduce` have the option to write output to a log file instead of printing it to the terminal. Use the cell below to set your log file or update it if you decide to start a new one:

In [ ]:
# To write output to a log file, uncomment the line below and fill in the path and file name for the log file
# logfile = ''
# Or, to just write output directly to terminal, uncomment the following line instead
logfile = None

Finally, import a few side-packages that will help with visualization in this notebook:

In [ ]:
from astropy.io import fits
from matplotlib import pyplot as P
import numpy as np
from mirac5reduce.utils.statfunc import medabsdev
import configparser, os

<a id="calc-meandark"></a>
## Calculate Mean Dark File

The function, `combine_frames.meanframe` calculates the mean frame for a range of file numbers and saves the output to a fits file with a populated header. While this function is general, it determines which raw file path to use from the config file from the data type specified when calling it.

In [ ]:
# Enter the full path where the raw dark files from the telescope are stored
raw_dark_path    = 

# Enter the start and end (inclusive) file numbers of the range of raw dark files that you want to average 
#     as integers
raw_dark_startno = 
raw_dark_endno   = 

# Enter the path where files created by the calibration scripts should be saved. The mean dark file saved 
#     here be named dark_[raw_dark_startno]_[raw_dark_endno].fits
calib_outpath = 

# Then calculate the mean dark frame. Turns on debugging mode to echo the parameters provided.
combine_frames.meanframe( config_file, 'dark', logfile = logfile, debug = True,
                          datapath = raw_dark_path, startno = raw_dark_startno, endno = raw_dark_endno, 
                          outpath = calib_outpath )

Use this cell to preview the mean dark frame saved to this file:

In [ ]:
## Generates file preview only; no editing needed ##
with fits.open( os.path.join(calib_outpath, 'dark_{0}_{1}.fits'.format(raw_dark_startno,raw_dark_endno) ) ) as hdulist:
    
    print('Mean Value: {0:.2e} DN'.format( np.mean(hdulist[0].data) ))
    print('Median Value: {0:.2e} DN'.format( np.median(hdulist[0].data) ))
    print('Std Dev: {0:.2e} DN'.format( np.std(hdulist[0].data) ))
    print('Med. Abs. Dev.: {0:.2e} DN'.format( medabsdev(hdulist[0].data) ))
    
    fig, axes = P.subplots(figsize = (15,7),  ncols=2)
    axes[0].imshow( hdulist[0].data, origin='lower', interpolation='nearest' )
    axes[0].set_title('Mean Dark', size=15)
    axes[0].set_xlabel('X [pix]', size=15)
    axes[0].set_ylabel('Y [pix]', size=15)
    axes[1].set_yscale('log')
    axes[1].hist(hdulist[0].data.ravel(), bins=500)
    axes[1].set_title('Mean Dark Pixel Values', size=15)
    axes[1].set_xlabel('Pixel Value [DN]', size=15)
    axes[1].set_ylabel('Number of Pixels', size=15)

<a id="calc-bpmask"></a>
## Create Bad Pixel Mask

The bad pixel mask is created with a simple sigma-filter on the median absolute deviation of the mean dark file created above.

In [ ]:
## This section uses the following variables set above when making the mean dark file. If not running sequentially,
##     uncomment these lines and set the relevant values here

## Start and end file numbers for dark files
# raw_dark_startno = 
# raw_dark_endno   = 

## Path where files created by calibration scripts are saved. Should already contain the mean dark file for the
##     above dark file numbers. Will create bad pixel mask file named 
##     bpmask_[raw_dark_startno]_[raw_dark_endno].fits
# calib_outpath = 

##

In [ ]:
# The threshold above and below which pixels are considered bad is 
#     this number x the M.A.D. on either side of the median
bp_threshold = 7.0

# Create the bad pixel mask file. Turns on debugging mode to echo the parameters provided.
bpmask.make_bpmask( config_file, logfile = logfile, debug = True,
                    startno = raw_dark_startno, endno = raw_dark_endno, bp_threshold = bp_threshold, 
                    outpath = calib_outpath )

While the histogram from the mean dark file above may provide more insight, the cell below can be used for a basic summary of the mask data.

In [ ]:
## Run cell for file preview only; no editing needed ##
with fits.open( os.path.join(calib_outpath,'bpmask_{0}_{1}.fits'.format(raw_dark_startno,raw_dark_endno)) ) as hdulist:
    
    print('Flagged {0} / {1} pixels ({2:.1f}%)'.format( np.sum(hdulist[0].data), hdulist[0].data.size, 
                                                       100. * np.sum(hdulist[0].data) / hdulist[0].data.size ))
    print('\n{0} ext 0 header:\n'.format('bpmask_{0}_{1}.fits'.format(raw_dark_startno,raw_dark_endno)))
    for key in hdulist[0].header.keys():
        print( '{0: <8} = {1: >24} / {2}'.format( *hdulist[0].header.cards[key] ) )
    

<a id="calc-flatfield"></a>
## Create Flatfield File

To calculate a dark subtracted and scaled mean flatfield frame, we need to use the `flatfield.create_flatfield` function:

In [ ]:
## This section uses the following variables set above when making the mean dark or bad pix mask file. If not 
##     running sequentially, uncomment these lines and set the relevant values here

## Start and end file numbers for dark files. Used to locate existing mean dark/bad pixel files made from them.
# raw_dark_startno = 
# raw_dark_endno   = 

## Path where files created by calibration scripts are saved. Should already contain the mean dark file and the
##     bad pixel mask created from the above dark file number range. Will create a flatfield fits file in this path
##     called flatfield_[raw_flat_startno]_[raw_flat_endno].fits
# calib_outpath = 

##

In [ ]:
# Enter the full path where the raw flat files from the telescope are stored
raw_flat_path    = 

# Enter the start and end file numbers (inclusive) of the range of raw flat files to use, as integers
raw_flat_startno = 
raw_flat_endno   = 

# Create the flatfield file. Turns on debugging mode to echo the parameters provided.
flatfield.create_flatfield( config_file, logfile = logfile, debug = True,
                            raw_flat_path = raw_flat_path, startno = raw_flat_startno, endno = raw_flat_endno, 
                            outpath = calib_outpath, dark_startno = raw_dark_startno, dark_endno = raw_dark_endno )


Use this cell to preview the mean flatfield file created and its fits header:

In [ ]:
## Generates file preview only; no editing needed ##
with fits.open( os.path.join(calib_outpath, 'flatfield_{0}_{1}.fits'.format(raw_flat_startno,raw_flat_endno) ) ) as hdulist:
    print('\n{0} ext 0 header:\n'.format('flatfield_{0}_{1}.fits'.format(raw_flat_startno,raw_flat_endno)))
    for key in hdulist[0].header.keys():
        print( '    {0: <8} = {1: >24} / {2}'.format( *hdulist[0].header.cards[key] ) )
    
    print('\nData Statistics:\n')
    print('  - Mean Value: {0:.2e} DN'.format( np.nanmean(hdulist[0].data) ))
    print('  - Median Value: {0:.2e} DN'.format( np.nanmedian(hdulist[0].data) ))
    print('  - Std Dev: {0:.2e} DN'.format( np.nanstd(hdulist[0].data) ))
    print('  - Med. Abs. Dev.: {0:.2e} DN'.format( medabsdev(hdulist[0].data) ))
    
    fig, axes = P.subplots(figsize = (15,7),  ncols=2)
    axes[0].imshow( hdulist[0].data, origin='lower', interpolation='nearest' )
    axes[0].set_title('Flatfield', size=15)
    axes[0].set_xlabel('X [pix]', size=15)
    axes[0].set_ylabel('Y [pix]', size=15)
    axes[1].set_yscale('log')
    axes[1].hist(hdulist[0].data.ravel(), bins=500)
    axes[1].set_title('Flatfield Pixel Values', size=15)
    axes[1].set_xlabel('(Normalized) Pixel Value [unitless]', size=15)
    axes[1].set_ylabel('Number of Pixels', size=15)

<a id="calc-meanflat"></a>
## Create Mean Flat File

Just as with the mean dark file, the `combine_frames.meanframe` function can be used to create a mean flat frame instead of a mean dark frame. Unlike `flatfield.create_flatfield` above, this is a simple mean of the frames provided, with no dark subtraction or scaling.

**Note**: The mean flat file is not currently used in the staring-mode reduction below. However, creating a mean flat file is an optional capability of this package, if desired.

In [ ]:
## This section uses the following variables set by the calibration sections above. If not 
##     running sequentially, uncomment these lines and set the relevant values here

## The full path where the raw flat files from the telescope are stored
# raw_flat_path    = 

## Path where files created by calibration scripts are saved. Will create a mean flat fits file in this path
##     called flat_[raw_flat_startno]_[raw_flat_endno].fits
# calib_outpath = 

##

In [ ]:
# Enter the start and end file numbers (inclusive) of the range of raw flat files that you want to average
# raw_flat_startno = 
# raw_flat_endno   = 

# Then calculate the mean flat frame. Turns on debugging mode to echo the parameters provided.
combine_frames.meanframe( config_file, 'flat', logfile = logfile, debug = True,
                          datapath = raw_flat_path, startno = raw_flat_startno, endno = raw_flat_endno, 
                          outpath = calib_outpath )

Again, the cell below just offers a preview of the contents of the created file:

In [ ]:
## Run for file preview only; no editing needed ##
with fits.open( os.path.join(calib_outpath,'flat_{0}_{1}.fits'.format(raw_flat_startno,raw_flat_endno)) ) as hdulist:
    
    print('Mean Value: {0:.2e} DN'.format( np.mean(hdulist[0].data) ))
    print('Median Value: {0:.2e} DN'.format( np.median(hdulist[0].data) ))
    print('Std Dev: {0:.2e} DN'.format( np.std(hdulist[0].data) ))
    print('Med. Abs. Dev.: {0:.2e} DN'.format( medabsdev(hdulist[0].data) ))
    
    fig, axes = P.subplots(figsize = (15,7),  ncols=2)
    axes[0].imshow( hdulist[0].data, origin='lower', interpolation='nearest' )
    axes[0].set_title('Mean Flat', size=15)
    axes[0].set_xlabel('X [pix]', size=15)
    axes[0].set_ylabel('Y [pix]', size=15)
    axes[1].set_yscale('log')
    axes[1].hist(hdulist[0].data.ravel(), bins=500)
    axes[1].set_title('Mean Flat Pixel Values', size=15)
    axes[1].set_xlabel('Pixel Value [DN]', size=15)
    axes[1].set_ylabel('Number of Pixels', size=15)

<a id="reduce-chopnod"></a>
## Reduce Chop/Nod Observations

Reduction of the chop nod data is currently limited to calculating the mean difference frame of all of the chop/nodded frames with `combine_frames.chopnodframe`.

It can be ran with the following:

In [ ]:
# The full path where the raw observation files are stored
raw_data_path = 

# Enter the start and end file numbers (inclusive) of the range of raw chop/nod files that you want to average
raw_data_startno = 
raw_data_endno   = 

# Set the chop and nod frequencies, in Hz
chopfreq = 
nodfreq  = 

# Enter the path where you want to save the reduced output files. Will create a file in this directory named
#     chopnod_[raw_data_startno]_[raw_data_endno].fits
reduce_outpath = 

# Then calculate and save the mean difference frame. Turns on debugging mode to echo the parameters provided.
combine_frames.chopnodframe( config_file, logfile = logfile, debug = True,
                             datapath = raw_data_path, startno = raw_data_startno, endno = raw_data_endno,
                             chopfreq = chopfreq, nodfreq = nodfreq, outpath = reduce_outpath )

The cell below provides a preview, with optional masking:

In [ ]:
# Set bool specifying whether you want to apply bad pixel mask created above to the plotted figures
plot_with_mask = False

# If you do set plot_with_mask = True, provide the path to the bad pixel mask that you wish to use, as well as
#     the file numbers used to create it. To use values set in earlier cells, just comment the next three lines 
#     out.
# calib_outpath    = 
# raw_dark_startno = 
# raw_dark_endno   =


## Run for image preview only; no editing needed ##
cn_data = fits.getdata( os.path.join(reduce_outpath,'chopnod_{0}_{1}.fits'.format(raw_data_startno,raw_data_endno)), 0 )
if plot_with_mask:
    mask = fits.getdata( os.path.join(calib_outpath,'bpmask_{0}_{1}.fits'.format(raw_dark_startno,raw_dark_endno))  , 0 )
    cn_data = np.ma.masked_array( cn_data, mask=mask )
    cn_data = cn_data.filled( np.nan )
fig, ax = P.subplots(figsize = (11,9))
im = ax.imshow( cn_data, origin='lower', interpolation='nearest' )
fig.colorbar(im, ax=ax)
ax.set_title('Mean Chop-Nod Subtracted Image', size=15)
ax.set_xlabel('X [pix]', size=15)
ax.set_ylabel('Y [pix]', size=15)


# Use this section to specify the x/y axis limits on the figure
# xlim = [ 462, 562 ]
# ylim = [ 462, 562 ]
# ax.set_xlim( xlim[0], xlim[1] )
# ax.set_ylim( ylim[0], ylim[1] )

If you know the chop/nod throw, in pixels, you can recombine the image here:

In [ ]:
# Assumes parallel chop/nod throws, where chop-1-nod-A and chop-2-nod-B are at the same location. 
# Specify the throw of one of the two negative chop/nod positions with respect 1A/2B position as [dx, dy], in pix, 
#    where a positive dx value indicates that the negative source imprint is to the right of the positive source
#    The other negative imprint will be assumed to be the opposite direction from the positive central source
offset = [ , ]

# combines frames, but does *not* trim around the positive source
frame_dy = cn_data.shape[0] - abs( 2*offset[1] )
frame_dx = cn_data.shape[1] - abs( 2*offset[0] )
x0 = abs(offset[0]); y0 = abs(offset[1])
recomb_frame =   cn_data[ y0 : frame_dy+y0, x0 : frame_dx+x0 ] - \
                 cn_data[ y0+offset[1] : frame_dy+y0+offset[1], x0+offset[0] : frame_dx+x0+offset[0] ] - \
                 cn_data[ y0-offset[1] : frame_dy+y0-offset[1], x0-offset[0] : frame_dx+x0-offset[0] ]
print('Recombined image array is (y,x) = {0}'.format(recomb_frame.shape))

# Plots recombined frame
fig, ax = P.subplots(figsize = (9,7))
im = ax.imshow( recomb_frame, origin='lower', interpolation='nearest' )
fig.colorbar(im, ax=ax)
ax.set_title('Re-combined Chop-Nodded Image', size=15)
ax.set_xlabel('X [pix]', size=15)
ax.set_ylabel('Y [pix]', size=15)

# Use this section to zoom into where your source is in the image
# xlim = [ 412, 512 ]
# ylim = [ 457, 557 ]
# ax.set_xlim( xlim[0], xlim[1] )
# ax.set_ylim( ylim[0], ylim[1] )

<a id="reduce-staring"></a>
## Reduce Staring Mode Observations

While not yet implemented in `mirac5reduce` in a fully automated way, staring mode observations can be performed by manually applying the calibration files created above.

First, **create a mean dark file, a bad pixel file, and a flatfield file** using the appropriate sections above. 

Note: It is assumed that all file types have the same integration time!

This reduction consists of the following steps:

1. Calculate the mean frame of your raw staring mode data. Saved to `reduce_outpath` as 'staring_raw_*.fits'.


2. Subtract the mean dark frame from the mean observation frame.


3. (Optional) Divide the dark-subtracted observation by the flatfield frame.


4. (Optional) Apply the bad pixel mask.


5. Save result to `reduce_outpath` as 'staring_reduced_*.fits'.

Use the following cell to specify which of the optional steps you'd like to include:

In [ ]:
# Set whether or not to perform step 3 above (dividing by flatfield) by setting do_flatfield to True/False
do_flatfield = True

# Set whether or not to perform step 4 above (applying the bad pixel mask) by setting do_bpmask to True/False
do_bpmask    = True

Then, use the following cell to set the parameters for your data:

In [ ]:
# Enter the path where the raw (observation) frame files are stored
raw_data_path = 

# Enter the start and end file numbers (inclusive) of the range of raw staring mode files
raw_data_startno = 
raw_data_endno   = 

# Path indicating where the output files created by these reduction scripts should be saved
reduce_outpath = 

# Enter the path where the (previously created) calibration files are stored. It must at least contain a mean
#     dark file, and can also contain the corresponding bad pixel mask file and an associated flatfield file.
#     To use value set in earlier cells, just comment out the following line:
calib_outpath    =  

# Enter the start and end file numbers of the dark files used to create the mean dark file and bad pixel mask in
#     the calib_outpath. To use value set in earlier cells, comment out the following two lines:
raw_dark_startno = 
raw_dark_endno   = 

# Enter the start and end file numbers of the flat files used to create the flatfield file in
#     the calib_outpath. To use value set in earlier cells, comment out the following two lines:
raw_flat_startno = 
raw_flat_endno   = 


In [ ]:
## Run this cell to perform the staring mode reduction ##
######## Should not require additional editing ##########

# Calculates mean frame of data
mean_rawdata_filename = 'staring_raw_{0}_{1}.fits'.format( raw_data_startno, raw_data_endno )
combine_frames.meanframe( config_file, 'obs', logfile = logfile, debug = True,
                          startno = raw_data_startno, endno = raw_data_endno, datapath = raw_data_path )



## Loads in raw mean file produced and manually applies dark subtraction and flatfield ##
print('\nDark subtraction:')
print('  - Mean staring mode frame: {0}'.format( os.path.join(reduce_outpath,mean_rawdata_filename) ))
raw_staring_frame = fits.getdata( os.path.join(reduce_outpath,mean_rawdata_filename), 0 )
print('  - Mean dark frame        : {0}'.format( os.path.join(calib_outpath,mean_dark_filename) ))
mean_dark_filename = 'dark_{0}_{1}.fits'.format(raw_dark_startno,raw_dark_endno)
mean_dark_frame   = fits.getdata( os.path.join( calib_outpath,mean_dark_filename), 0 )
red_staring_frame = raw_staring_frame - mean_dark_frame


## Divides by the flatfield, if requested ##
print('Flatfield division: {0}'.format(do_flatfield))
if do_flatfield:
    flatfield_filename = 'flatfield_{0}_{1}.fits'.format(raw_flat_startno,raw_flat_endno)
    print('  - Mean flatfield frame   : {0}'.format( os.path.join(calib_outpath,flatfield_filename) ))
    flatfield_frame   = fits.getdata( os.path.join( calib_outpath,flatfield_filename), 0 )
    red_staring_frame /= flatfield_frame
else:
    flatfield_filename = None


## Applies the bad pixel mask, if requested ##
print('BP mask application: {0}'.format(do_bpmask))
if do_bpmask:
    bp_filename       = 'bpmask_{0}_{1}.fits'.format(raw_dark_startno,raw_dark_endno)
    print('  - Bad Pix Mask File     : {0}'.format( os.path.join(calib_outpath,bp_filename) ))
    bpmask_frame      = fits.getdata( os.path.join( calib_outpath,bp_filename), 0 )
    red_staring_frame = np.ma.masked_array( red_staring_frame, mask = bpmask_frame )
    red_staring_frame = red_staring_frame.filled( np.nan )
else:
    bp_filename       = None


## Saves the results to a file ##
staring_data_filename = 'staring_reduced_{0}_{1}.fits'.format( raw_data_startno, raw_data_endno )
print('Saving reduced frame: {0}'.format( os.path.join(reduce_outpath,staring_data_filename) ))
hdu = fits.PrimaryHDU( red_staring_frame )
hdu.header['FILETYPE'] =   'Staring Reduct'
hdu.header['NFRAMES' ] = ( raw_data_endno-raw_data_startno , 'Number raw flat frames used'         )
hdu.header['STARTNO' ] = ( raw_data_startno                , 'First file number of obs data range' )
hdu.header['ENDNO'   ] = ( raw_data_endno                  , 'Last file number of obs data range'  )
hdu.header['DARKFILE'] = ( mean_dark_filename              , 'Dark file used'                      )
hdu.header['MASKFILE'] = ( bp_filename                     , 'Bad Pixel Mask file used'            )
hdu.header['FLFDFILE'] = ( flatfield_filename              , 'Flatfield file used'                 )
hdu.writeto( os.path.join( reduce_outpath, staring_data_filename ) )

In [ ]:
## Use this cell to visualize the reduced data created ##

# Plots reduced data 
fig, ax = P.subplots(figsize = (9,7))
im = ax.imshow( red_staring_frame, origin='lower', interpolation='nearest' )
fig.colorbar(im, ax=ax)
ax.set_title('Reduced Staring-Mode Image', size=15)
ax.set_xlabel('X [pix]', size=15)
ax.set_ylabel('Y [pix]', size=15)

# Use this section to zoom into where your source is in the image
# xlim = [ 450, 580 ]
# ylim = [ 450, 580 ]
# ax.set_xlim( xlim[0], xlim[1] )
# ax.set_ylim( ylim[0], ylim[1] )